# JSBSim L-4S-5 Flight Recreation

Run the model in JSBSim and compare results to technical papers.

Download a copy of the Lambda-4S JSBSim configuration:

 - [JSBSim_Lambda-4S.zip](JSBSim_Lambda-4S.zip)

In [ ]:
# dependencies
from numpy import loadtxt
import matplotlib.pyplot as plt
%matplotlib inline

blue = "#3366cc"
red = "#e31d1d"
green =  "#76e146"

g_0 = 9.80665

LB2KG = 0.453592
IN2M = 0.0254
SF2KGM = 1.35581795

# Historical data
columns = loadtxt('../data/traj_model_case_1_booster.csv', delimiter=',', unpack=True)
case_1_booster_d, case_1_booster_alt = columns
columns = loadtxt('../data/traj_model_case_2_booster.csv', delimiter=',', unpack=True)
case_2_booster_d, case_2_booster_alt = columns
columns = loadtxt('../data/traj_model_case_3_booster.csv', delimiter=',', unpack=True)
case_3_booster_d, case_3_booster_alt = columns
columns = loadtxt('../data/traj_model_case_1_stage1.csv', delimiter=',', unpack=True)
case_1_stage1_d, case_1_stage1_alt = columns
columns = loadtxt('../data/traj_model_case_2_stage1.csv', delimiter=',', unpack=True)
case_2_stage1_d, case_2_stage1_alt = columns
columns = loadtxt('../data/traj_model_case_3_stage1.csv', delimiter=',', unpack=True)
case_3_stage1_d, case_3_stage1_alt = columns

stage1_thrust_time, stage1_thrust = loadtxt('../data/thrust_booster_and_stage1.csv', delimiter=',', unpack=True)

# Import sim data
columns = loadtxt("JSBSim/data.csv", delimiter=',', unpack=True, comments="T")

times              = columns[ 0]
MET                = columns[ 1]
altitudeMsl        = columns[ 2]
latitude           = columns[ 3]
longitude          = columns[ 4]
thrust             = columns[ 5] * 4.44822   # to newtons
total_mass         = columns[ 6] * 0.453592  # to kg
pitch              = columns[ 7]
ecei_vmag          = columns[ 8] * 0.3048    # to m/s
eci_vmag           = columns[ 9] * 0.3048    # to m/s
qbar               = columns[10] * 4.882e-4  # to kg/cm2
acc_u              = columns[11] * 0.3048    # to m/s2
acc_v              = columns[12] * 0.3048    # to m/s2
acc_w              = columns[13] * 0.3048    # to m/s2
downrange          = columns[14]
alpha              = columns[15]
roll_rate          = columns[16] * 57.2958   # to deg/s

## Mass Table

This is the mass and moment of inertia table output by JSBSim's initilization report:

In [ ]:
masstable = False
header = ["Vehicle Component", "Weight [kg]", "CG-X [m]", "CG-Z [m]", u"Ixx [kg·m²]", u"Iyy [kg·m²]", u"Izz [kg·m²]"]

print(" |".join(["%20s" % name for name in header]))
print(" |".join([" :------------------"] + [" ------------------:" for name in header[1:]]))

def test_num(s):
    try:
        float(s)
    except:
        return False
    return True

with open("JSBSim/simrun.txt") as tablein:
    for line in tablein.readlines():
        if masstable:
            line = line.replace('-', ' -')

            splitonctrl = line.split('\x1b')
            if len(splitonctrl) < 3:
                continue
            
            name = splitonctrl[1].replace('[1m', '').strip()
            if "Total" in name:
                masstable = False
                name = "**Totals:**"
                print(" |".join([" -------------------" for name in header]))
                values = splitonctrl[1].split()[2:4] + splitonctrl[1].split()[5:9] 
            else:
                values = splitonctrl[2].split()[1:3] + splitonctrl[2].split()[4:8]
            
            if len(values) < 3:
                continue
                
            values[0] = "%0.1f" % (float(values[0]) * LB2KG)
            
            values[1] = "%0.3f" % (float(values[1]) * IN2M)
            values[2] = "%0.3f" % (float(values[2]) * IN2M)
            
            values[3] = "%0.1f" % (float(values[3]) * SF2KGM)
            values[4] = "%0.1f" % (float(values[4]) * SF2KGM)
            values[5] = "%0.1f" % (float(values[5]) * SF2KGM)
            
            row = [name] + values
            print(" |".join(["%20s" % value for value in row]))

        if "Weight    CG-X    CG-Y    CG-Z         Ixx         Iyy         Izz         Ixy         Ixz         Iyz" in line:
            masstable = True



## First Stage Thrust

Total thrust of the first stage and strap-on boosters compared to the published L-4S-2 vehicle data.

In [ ]:
fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S-5 First Stage Total Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(stage1_thrust_time, (stage1_thrust*g_0)/1000.0, color=blue, alpha=0.4, label="L-4S-2 Vehicle Data")
plt.plot(MET, thrust/1000.0, color=red, alpha=0.8, label=u"JSBSim Simulation")

#plt.ylim([0,5])
plt.xlim([0,30])
ax1.legend(loc=1)
plt.show()

## First 5 km Altitude/Downrage Plot

In [ ]:
fig, ax1 = plt.subplots(figsize=(6,10))
plt.title(r"L-4S-5 Booster Altitude/Downrage")
plt.ylabel(r"Altitude [km]")
plt.xlabel(r"Downrange [km]")

#plt.fill_between(case_1_booster_d, 0, case_1_booster_alt, color=blue, alpha=0.10)
#plt.fill_between(case_3_booster_d, 0, case_3_booster_alt, color='white')
plt.fill_between(case_3_stage1_d, 0, case_3_stage1_alt, color=blue, alpha=0.05)
plt.fill_between(case_1_stage1_d, 0, case_1_stage1_alt, color='white')


plt.plot(case_1_booster_d, case_1_booster_alt, color=blue, lw=1, alpha=0.3, label=u"Case 1: Launch Angle=62.0°")
plt.plot(case_2_booster_d, case_2_booster_alt, color=blue, lw=1, alpha=0.4, label=u"Case 2: Launch Angle=64.0°")
plt.plot(case_3_booster_d, case_3_booster_alt, color=blue, lw=1, alpha=0.3, label=u"Case 3: Launch Angle=66.0°")
plt.plot(case_1_stage1_d, case_1_stage1_alt, color=blue, lw=1, alpha=0.3)
plt.plot(case_2_stage1_d, case_2_stage1_alt, color=blue, lw=1, alpha=0.4)
plt.plot(case_3_stage1_d, case_3_stage1_alt, color=blue, lw=1, alpha=0.3)

plt.plot(downrange/1000.0, (altitudeMsl)/1000.0, color=red, lw=1.5, alpha=0.8, label="JSBSim Simulation")


plt.axis('equal')
plt.ylim([0,5])
plt.xlim([0,3])
plt.xticks(range(4))
ax1.legend(loc=2)
plt.show()